In [1]:
#!pip install scikit-learn

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import TargetEncoder

In [2]:
data_train = pd.read_csv('../data_cleaned/data_train.csv')
item_categories = pd.read_csv('../data_cleaned/item_categories.csv')
items = pd.read_csv('../data_cleaned/items.csv')
shops = pd.read_csv('../data_cleaned/shops.csv')

In [3]:
#merged = data_train.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left')

In [4]:
def parse_city(shop_name):
    if shop_name.split()[0] == '!Якутск':
        return  'Якутск'

    if shop_name.split()[0] == 'Сергиев':
            return  'Сергиев Посад'
    else:
        return shop_name.split()[0]

In [5]:
#dates = pd.to_datetime(data_train['date'], format='%d.%m.%Y')


In [6]:
import calendar

def number_to_month(numbers):
    # Create a list of month names using the calendar module
    month_names = list(calendar.month_name)[1:]  # Extract months from 1 to 12 (exclude the empty element at index 0)
    
    # Convert the numbers to month names
    result = [month_names[num % 12] for num in numbers]  # Use num % 12 to handle numbers beyond 11
    
    return result


In [7]:

def number_to_month(numbers):
    result = [num %12 for num in numbers]  # Use num % 12 to handle numbers beyond 11
    
    return result


In [8]:
def convert_to_year(month_num):
    # January 2013 is the starting point
    start_year = 2013
    
    # Calculate the year
    year = start_year + (month_num // 12)
    
    return year

In [9]:

def create_lags(data,item_lags=None,price_lags=None, test_included=False):
    if test_included:
        lagged_items = data[data['date_block_num'] < 34] [['date_block_num','item_cnt_month','shop_id','item_id']].copy()
        lagged_prices = data[data['date_block_num'] < 34] [['date_block_num','avg_item_price','shop_id','item_id']].copy()
    else:
        lagged_items = data[['date_block_num','item_cnt_month','shop_id','item_id']].copy()
        lagged_prices = data[['date_block_num','avg_item_price','shop_id','item_id']].copy()
    for lag in item_lags:
        lagged_items['date_block_num']+=lag#previous month becomes present
        data=data.merge(lagged_items, how='left',on=['date_block_num','shop_id','item_id'], suffixes=('', f'_lag_{lag}'))
        lagged_items['date_block_num']-=lag

    
    for lag in price_lags:
        lagged_prices['date_block_num']+=lag#previous month becomes present
        data=data.merge(lagged_prices, how='left',on=['date_block_num','shop_id','item_id'], suffixes=('', f'_lag_{lag}'))
        lagged_prices['date_block_num']-=lag

        
    return data

In [10]:
#def encode_numerical_features(data, features):#Best way?? May be used for linear models
#    te = TargetEncoder(target_type='continuous')
#    transformed=te.fit_transform(data[features],y=data['item_cnt_month'])
#    for idx,feature in enumerate(features):
#        data[feature+'_trg_enc'] = transformed[:,idx]
#    return data

In [11]:
def find_mean_price(lagged):
    return lagged[[f'avg_item_price_lag_{lag}' for lag in price_lags[:6]]].mean(axis=1)
    

In [12]:
def find_mean_cnt(lagged):
    return lagged[[f'item_cnt_month_lag_{lag}' for lag in item_lags[:6]]].mean(axis=1)
    

In [13]:
def avg(data):
    v = data['item_cnt_month'].mode().mean()

    return v#.mode()[0]#.mean()

In [14]:
def avg_by_city(data, city):
    v = data [data['city']==city] ['item_cnt_month'].mode().mean()
    
    return v# .mode()[0]

In [15]:
def avg_by_shop(data, shop):
    v = data [data['shop_id']==shop] ['item_cnt_month'].mode().mean()
    
    return v# .mode()[0]#.mean()

In [16]:

def create_lags_columns(data, columns,item_lags=[1,2,3],price_lags=[1,2,3], test_included=False):
    if test_included:
        lagged_items = data [data['date_block_num'] < 34]  [[*columns,'item_cnt_month']] 
        lagged_prices = data [data['date_block_num'] < 34]  [[*columns,'avg_item_price']]
    else:
        lagged_items = data[[*columns,'item_cnt_month']]
        lagged_prices = data[[*columns,'avg_item_price']]
    
    for lag in item_lags:
        lagged_items['date_block_num']+=lag#previous month becomes present

        averaged = lagged_items.groupby(columns).mean().reset_index()#future 
        
        data=data.merge(averaged, how='left',on=columns, suffixes=('', f'{'_'.join(columns[1:])}_lag_{lag}'))
        
        lagged_items['date_block_num']-=lag

    
    for lag in price_lags:
        lagged_prices['date_block_num']+=lag#previous month becomes present

        averaged = lagged_prices.groupby(columns).mean().reset_index()
        
        data=data.merge(averaged, how='left',on=columns, suffixes=('', f'{'_'.join(columns[1:])}_lag_{lag}'))
        lagged_prices['date_block_num']-=lag

        
    return data

In [17]:
import pandas as pd

# Assuming your DataFrame is named 'df'

# Function to calculate EMA over the lag columns
def calculate_ema_3(df, min_periods=3, target='item_cnt_month'):
    #f'{'_'.join(columns[1:])}_lag_{lag}')
    # Create a temporary dataframe with the 3 lags
    def calculate_ema(df,group_col):
        lag_data = df[[f'{target}{group_col}_lag_1', f'{target}{group_col}_lag_2', f'{target}{group_col}_lag_3']]
        
        # Compute the row-wise EMA for the 3 lag columns using ewm
        ema_series = lag_data.ewm(alpha=0.5, min_periods=min_periods, axis=1).mean().mean(axis=1)
        
        return ema_series

    
    # Calculate EMA for 'item_category_name'
    df[f'ema_{target}_item_category_name'] = df.groupby('item_category_name').apply(lambda group: calculate_ema(group, 'item_category_name')).reset_index(drop=True)
    
    # Calculate EMA for 'item_id'
    df[f'ema_{target}_item_id'] = df.groupby('item_id').apply(lambda group: calculate_ema(group, 'item_id')).reset_index(drop=True)
    
    # Calculate EMA for 'super_category'
    df[f'ema_{target}_super_category'] = df.groupby('super_category').apply(lambda group: calculate_ema(group, 'super_category')).reset_index(drop=True)
    
    # Calculate EMA for 'city'
    df[f'ema_{target}_city'] = df.groupby('city').apply(lambda group: calculate_ema(group, 'city')).reset_index(drop=True)
    
    # The df will now have 4 new EMA columns based on the lag values

    return df


In [18]:
#def choose_previos_blocks(data_SORTED, idx):
#    #idx - idx in data_SORTED
#    block = data_SORTED.iloc[idx]['date_block_num']
#    last_index_block = blocks[block-1]
#    return data_SORTED.iloc[0:last_index_block]

In [19]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor

# Assuming 'df' is your DataFrame
c1=0
c2=0
# Function to apply KNN and compute the weighted mean for item_cnt_month
def knn_weighted_mean(df, group_cols, target_col=None, price_col=None, date_block_num_col=None, n_neighbors=5,window_size=3):
    """
    window_size - in which date block nums to search for nearest neigbours
    """
    global c1
    global c2
    # Initialize a column for storing the results
    
    
    #results=pd.DataFrame(np.nan, index = np.arange(len(df)), columns = ['A'])
    # Iterate through each unique date_block_num (k)
    #to_predict = df.iloc[first_index_block_3:]
    
    results = np.full(len(df), np.nan)
    
    for date_block_num_k in df[date_block_num_col].unique():
        
        # Filter the data to include only rows with date_block_num < k and ( date_block_num_k - df[date_block_num_col] ) <= window_size
        #print(date_block_num_k - df[date_block_num_col] )
        prev  = df[ (df[date_block_num_col] < date_block_num_k) & \
        (( date_block_num_k - df[date_block_num_col] ) <= window_size) & \
        (df['date_block_num_diff'] >=2)]
        
        if len(prev)==0:
            #results[mask]=np.nan
            c1+=1
            #print('No previous blocks')
            continue
        
        # Apply grouping criteria and filter for group-specific calculations
        for group_val in df[group_cols].drop_duplicates().values:
            
            mask = (prev[group_cols] == group_val).all(axis=1) & \
            (~ np.squeeze (np.array(( np.isnan(prev[price_col ]) ))))
            
            pred_mask = ((df[group_cols] == group_val).all(axis=1)) & \
            (df['date_block_num']==date_block_num_k)  & \
            (~ np.squeeze (np.array(( np.isnan(df[price_col ]) ))))
            
            X = prev.loc[mask, [price_col]].values
            
            y = prev.loc[mask, target_col].values

            X_test=pd.DataFrame( df[pred_mask][price_col] ).values
            
            if len(X) == 0:
                c1+=1
                #print('ema is nan')
                continue

            if len(X) < n_neighbors:
                c1+=1
                results[pred_mask] = y.mean()
                #print('not enough neighb, avg')
                
                #print('X < n_neighbors')
                continue

            if len(X_test) == 0:
                c1+=1
                #print('no such in test')
                continue

            
            knn = KNeighborsRegressor(n_neighbors=n_neighbors)#, weights='distance'
            knn.fit(X, y)

            
            

            #print(results[pred_mask])
            results[pred_mask] = knn.predict( X_test )
            c2+=1
        print('block proc:',date_block_num_k)
    
    return results


In [21]:
def find_means_last6(lagged):
    means = lagged[['item_cnt_month_lag_1', 
                 'item_cnt_month_lag_2',
                 'item_cnt_month_lag_3',
                 'item_cnt_month_lag_4', 
                 'item_cnt_month_lag_5', 
                 'item_cnt_month_lag_6'
                   ]].mean(axis=1).rename('avg')
    
   
    m_i = pd.concat([lagged[['item_id','date_block_num']], means],axis=1)
    m_i = m_i.groupby(['item_id','date_block_num'])['avg'].transform('mean')
    lagged['mean_price_month_item_id'] = m_i

    m_i = pd.concat([lagged[['item_category_id','shop_id','date_block_num']], means],axis=1)
    m_i = m_i.groupby(['item_category_id','shop_id','date_block_num'])['avg'].transform('mean')
    lagged['mean_price_month_item_categoty_id_shop_id'] = m_i

    m_i = pd.concat([lagged[['item_category_id','date_block_num']], means],axis=1)
    m_i = m_i.groupby(['item_category_id','date_block_num'])['avg'].transform('mean')
    lagged['mean_price_month_item_categoty_id'] = m_i
    
    return lagged
    #means_item = means
    

In [22]:
def transform(data_train, item_categories,items,shops, test_included=True ):
    grouped=data_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_price':'mean',
                                                                    'item_cnt_day':'sum'
                                                                    })#take some (a lot)))) ) time

    print('1st grouping done')
    grouped=grouped.reset_index()
    grouped = grouped.sort_values(by='date_block_num')
    merged = grouped.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left')
    print('merged created')
    merged['month'] = number_to_month(merged['date_block_num'])
    merged['year'] = merged['date_block_num'].apply(convert_to_year)
    merged['super_category'] = merged['item_category_name'].apply(lambda a: a.split()[0])
    merged['city'] = merged['shop_name'].apply(parse_city)
    merged=merged.rename(columns={'item_price':'avg_item_price','item_cnt_day':'item_cnt_month'})
    item_lags=list([*range(1,7),12,13,24])
    price_lags=list([*range(1,7),12,24])
    
    lagged = create_lags(merged,item_lags=item_lags,price_lags=price_lags,test_included=test_included )#takes a bit time
    print('lags created')
    lagged['shop_id_cat'] = lagged['shop_id'].astype('str')
    lagged['item_id_cat'] = lagged['item_id'].astype('str')
    lagged['item_category_id_cat'] = lagged['item_category_id'].astype('str')
    #lagged = encode_numerical_features(lagged, features=['item_category_id_cat', 'super_category','city'])#this part may not be used
    df = lagged

    df = find_means_last6(lagged)

    print('means for last 6 mothes found')
    
    # 1. Create the 'first_date_block_num' column (first appearance of each item_id)
    df['first_date_block_num'] = df.groupby('item_id')['date_block_num'].transform('min')
    df['date_block_num_diff'] = df['date_block_num'] - df['first_date_block_num']
    # 2. Create the 'first_cat_block_num' column (first appearance of each item_category_id)
    #df['first_cat_block_num'] = df.groupby('item_category_id')['date_block_num'].transform('min')
    #df['first_sup_cat_block_num'] = df.groupby('super_category')['date_block_num'].transform('min')

    df = create_lags_columns(df,columns=['date_block_num','super_category'],test_included=test_included )
    df = create_lags_columns(df,columns=['date_block_num','item_id'],test_included=test_included )
    df = create_lags_columns(df,columns=['date_block_num','city'] )
    print('column lags created')
    df = create_lags_columns(df,columns=['date_block_num','item_category_name'] )
    df = calculate_ema_3(df, target='avg_item_price')
    print('1st EMA calculated')
    df = calculate_ema_3(df, target='item_cnt_month')
    print('2nd EMA calculated')
    group_cols=['item_category_name']

    first_index_block_3 = df[df['date_block_num'] == 3 ].index[0]
    group_cols=['item_category_name']

    #first_index_block_3 = df[df['date_block_num'] == 3 ].index[0]
    #blocks = [df[df['date_block_num'] == block-1 ].index[-1] for block in np.sort(df.date_block_num.unique())[1:]]
    df = df.drop([
        'item_cnt_monthsuper_category_lag_1',
        'item_cnt_monthsuper_category_lag_2',
        'item_cnt_monthsuper_category_lag_3',
        'avg_item_pricesuper_category_lag_1',
        'avg_item_pricesuper_category_lag_2',
        'avg_item_pricesuper_category_lag_3',
        'item_cnt_monthitem_id_lag_1',
        'item_cnt_monthitem_id_lag_2', 
        'item_cnt_monthitem_id_lag_3',
        'avg_item_priceitem_id_lag_1',
        'avg_item_priceitem_id_lag_2',
        'avg_item_priceitem_id_lag_3',
        'item_cnt_monthcity_lag_1',
        'item_cnt_monthcity_lag_2', 
        'item_cnt_monthcity_lag_3',
        'avg_item_pricecity_lag_1', 
        'avg_item_pricecity_lag_2',
        'avg_item_pricecity_lag_3', 
        'item_cnt_monthitem_category_name_lag_1',
        'item_cnt_monthitem_category_name_lag_2',
        'item_cnt_monthitem_category_name_lag_3',
        'avg_item_priceitem_category_name_lag_1',
        'avg_item_priceitem_category_name_lag_2',
        'avg_item_priceitem_category_name_lag_3',
        
        #'first_date_block_num',
        #'shop_id',
        #'item_id',
        #'shop_name',
        #'item_category_id',
        #'item_category_name',
        
    ], axis=1)
    return df[df['date_block_num'] >= 3 ]
    #to delete - lag_2,lag_3, lags for columns
            

In [23]:
#df = transform(data_train, item_categories,items,shops )

In [24]:
test = pd.read_csv('../data_cleaned/test.csv')

In [25]:
test['date_block_num'] = 34

In [26]:
test_trans=transform(pd.concat([data_train,test]),  item_categories,items,shops,test_included=True )

1st grouping done
merged created
lags created
means for last 6 mothes found


/tmp/ipykernel_31902/1287334314.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']+=lag#previous month becomes present
/tmp/ipykernel_31902/1287334314.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']-=lag
/tmp/ipykernel_31902/1287334314.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

column lags created


/tmp/ipykernel_31902/1287334314.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']-=lag
/tmp/ipykernel_31902/1287334314.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']+=lag#previous month becomes present
/tmp/ipykernel_31902/1287334314.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

1st EMA calculated


/tmp/ipykernel_31902/3532761222.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f'ema_{target}_item_category_name'] = df.groupby('item_category_name').apply(lambda group: calculate_ema(group, 'item_category_name')).reset_index(drop=True)
/tmp/ipykernel_31902/3532761222.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f'ema_{target}_item_id'] = df.groupby('item_id').apply(lambda group: calcula

2nd EMA calculated


In [27]:
test_trans.columns

Index(['shop_id', 'item_id', 'date_block_num', 'avg_item_price',
       'item_cnt_month', 'item_name', 'item_category_id', 'item_category_name',
       'shop_name', 'month', 'year', 'super_category', 'city',
       'item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3',
       'item_cnt_month_lag_4', 'item_cnt_month_lag_5', 'item_cnt_month_lag_6',
       'item_cnt_month_lag_12', 'item_cnt_month_lag_13',
       'item_cnt_month_lag_24', 'avg_item_price_lag_1', 'avg_item_price_lag_2',
       'avg_item_price_lag_3', 'avg_item_price_lag_4', 'avg_item_price_lag_5',
       'avg_item_price_lag_6', 'avg_item_price_lag_12',
       'avg_item_price_lag_24', 'shop_id_cat', 'item_id_cat',
       'item_category_id_cat', 'mean_price_month_item_id',
       'mean_price_month_item_categoty_id_shop_id',
       'mean_price_month_item_categoty_id', 'first_date_block_num',
       'date_block_num_diff', 'ema_avg_item_price_item_category_name',
       'ema_avg_item_price_item_id', 'ema_avg_item

In [29]:
test_trans.to_csv('../data.csv')

In [28]:
test_trans.query('date_block_num==34')

,shop_id,item_id,date_block_num,avg_item_price,item_cnt_month,item_name,item_category_id,item_category_name,shop_name,month,...,item_category_id_cat,date_block_num_diff,ema_avg_item_price_item_category_name,ema_avg_item_price_item_id,ema_avg_item_price_super_category,ema_avg_item_price_city,ema_item_cnt_month_item_category_name,ema_item_cnt_month_item_id,ema_item_cnt_month_super_category,ema_item_cnt_month_city
1608225,59,22166,34,NaN,0.0,Язык запросов 1С:Предприятия [Цифровая версия],54,Книги - Цифра,"Ярославль ТЦ ""Альтаир""",10,...,54,25,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
1608226,59,22164,34,NaN,0.0,ЯРОСТЬ (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",10,...,37,9,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
1608227,59,22163,34,NaN,0.0,ЯРОСТЬ ( регион),40,Кино - DVD,"Ярославль ТЦ ""Альтаир""",10,...,40,3,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
1608228,59,22162,34,NaN,0.0,ЯРОСТЬ,40,Кино - DVD,"Ярославль ТЦ ""Альтаир""",10,...,40,16,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
1608229,2,32,34,NaN,0.0,1+1,40,Кино - DVD,"Адыгея ТЦ ""Мега""",10,...,40,34,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822420,2,42,34,NaN,0.0,100 Best romantic melodies (mp3-CD) (Digipack),57,Музыка - MP3,"Адыгея ТЦ ""Мега""",10,...,57,34,139.127975,NaN,139.127975,990.606128,3.014701,NaN,3.014701,1.930493
1822421,2,38,34,NaN,0.0,10 САМЫХ ПОПУЛЯРНЫХ КИНОКОМЕДИЙ ХХ ВЕКА 10DVD...,41,Кино - Коллекционное,"Адыгея ТЦ ""Мега""",10,...,41,16,139.127975,NaN,139.127975,990.606128,3.014701,NaN,3.014701,1.930493
1822422,2,33,34,NaN,0.0,1+1 (BD),37,Кино - Blu-Ray,"Адыгея ТЦ ""Мега""",10,...,37,34,139.127975,NaN,139.127975,990.606128,3.014701,NaN,3.014701,1.930493
1822423,59,22167,34,NaN,0.0,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,49,Книги - Методические материалы 1С,"Ярославль ТЦ ""Альтаир""",10,...,49,26,139.127975,NaN,139.127975,990.606128,3.014701,NaN,3.014701,1.930493


In [30]:
test_trans.query('date_block_num==34') .isna().sum()

shop_id                                       0
item_id                                       0
date_block_num                                0
avg_item_price                           214200
item_cnt_month                                0
item_name                                     0
item_category_id                              0
item_category_name                            0
shop_name                                     0
month                                         0
year                                          0
super_category                                0
city                                          0
item_cnt_month_lag_1                     185528
item_cnt_month_lag_2                     187735
item_cnt_month_lag_3                     185251
item_cnt_month_lag_4                     187761
item_cnt_month_lag_5                     189988
item_cnt_month_lag_6                     191029
item_cnt_month_lag_12                    191776
item_cnt_month_lag_13                   

In [29]:
#set_y = set(map(tuple, test_trans.query('date_block_num==33') [['shop_id','item_id']].values))
#idx = [tuple(point) not in set_y for point in test_trans.query('date_block_num==34') [['shop_id','item_id']].values]

#test_trans.query('date_block_num==34') [['shop_id','item_id']].values[idx].shape

In [27]:
!pwd

/home/valeriy/python_projects/predict_future_sales/EDA
